In [1]:
import pandas as pd
import os

from sqlalchemy import create_engine, text

#maximum number of rows to display
pd.options.display.max_rows = 20

In [4]:
import sqlite3

In [6]:
conexion = sqlite3.connect('kavak_db_v1_0_1.sql')

In [7]:
cursor = conexion.cursor()

In [8]:
cursor.execute("SELECT * FROM kavak_db_v1_0_1")

DatabaseError: file is not a database

In [11]:
import sqlite3

# Crea un objeto de conexión a la base de datos SQLite
con = sqlite3.connect("kavak_db_v1_0_1.sql")

# Con la conexión, crea un objeto cursor
cur = con.cursor()

# El resultado de "cursor.execute" puede ser iterado por fila
for row in cur.execute('SELECT * FROM species;'):
    print(row)

DatabaseError: file is not a database